In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.14.0 pyspark-shell'

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read.format('xml').options(rowTag='page').load('hdfs:/enwiki_small.xml')

:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7568939b-68d7-44e7-a869-9d647c8dcb8e;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.14.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 305ms :: artifacts dl 8ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.14.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.2.5 from central in [default]
	org.glassfish.jaxb#txw2;2.3.4 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|

In [3]:
import regex
from pyspark.sql.types import StringType,ArrayType
from pyspark.sql.functions import udf, col,explode


def clean_title(titles):
    #All the letters should be convert to lower case.
    return titles.lower()

def parse(match_):
    ret = None
    match_ = match_.split("|")
    for ele in match_:
        if "#" in ele:
            continue
        elif (":" in ele) and (not ele.startswith("Category:")):
            continue
        else:
            ret = ele
            break
            
    if ret:
        return ret.lower()
    else:
        return ret

def clean_text(revisions):
    #All the letters should be convert to lower case.
    text = revisions["text"]['_VALUE']
    links_ = []
    #If multiple links appear in the brackets, take the first one.(Ignore links that contain a #)
    matches = regex.findall(r'\[\[((?:[^[\]]+|(?R))*+)\]\]', text)
    for match in matches:
            link = parse(match)
            if link:
                links_.append(link)
    if len(links_) > 0:
        return links_
    else:
        return None
    
           
clean_title_udf = udf(lambda k: clean_title(k), StringType())
clean_text_udf = udf(lambda k: clean_text(k), ArrayType(StringType()))


output = df.withColumn("title",clean_title_udf(col("title"))).\
withColumn("links", clean_text_udf(col("revision"))).\
select("title",explode("links")).withColumnRenamed("col","links")

output.filter(output.links.isNotNull()).repartition(1).orderBy(["title","links"]).limit(5).\
write.csv("gs://csee4121homework/outputs/p1t2.csv",mode="overwrite",sep = "\t")